# Imports and Installs



In [ ]:
import math
import random
import os
import time
import datetime as dt
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import transformers
from transformers import AutoTokenizer, AutoModel
from sklearn.model_selection import train_test_split
from sentence_transformers import SentenceTransformer
import tqdm
import importlib
from pathlib import Path
from datasets import Dataset, DatasetDict, concatenate_datasets
from transformers import AutoTokenizer
from itertools import chain

# Initial Models

We will build the following modules:

1. `ChunkwiseLinearAttentionHead`: encapsulates the linear attention mechanism, computed using a chunkwise parallel algorithm.

2. `CLAWrapperNextToken`: wrapper around a _single_ ChunkwiseLinearAttentionHead for next-token prediction. Wrapper consists of an embedding layer to convert tokens into vectors for the head, the head itself, and 1 fully-connected layer to project onto token probabilities.

3. `CLAWrapperTextClassification`: wrapper around a _single_ ChunkwiseLinearAttentionHead for text classification. Similar architecture as `CLAWrapperNextToken`, except the final linear layer output dimension is `num_classes` instead of `vocab_size`.

In [ ]:
# class Parallel

In [ ]:
class ChunkwiseLinearAttentionHead(nn.Module):

  def __init__(self, in_features, out_features, chunk_size, seq_len):
    super().__init__()
    self.in_features = in_features
    self.out_features = out_features
    self.chunk_size = chunk_size
    self.seq_len = seq_len

    self.Q_linear = nn.Linear(in_features, out_features, bias=False)
    self.K_linear = nn.Linear(in_features, out_features, bias=False)
    self.V_linear = nn.Linear(in_features, out_features, bias=False)

    M = torch.tensor(torch.tril(torch.ones(chunk_size, chunk_size)))
    self.register_buffer("M", M)


    # self.rope_rotator

  def rope(self):
    pass

  def forward(self, x):
    # Calculate number of chunks we need
    n_chunks = x.shape[1] // self.chunk_size

    # Compute initial Q, K, V
    Q = self.Q_linear(x)
    K = self.K_linear(x)
    V = self.V_linear(x)

    # Sanity check
    # print(Q.shape)
    # print(K.shape)
    # print(V.shape)

    # RoPE reshaping -- pair adjacent values in Q and K
    Q = Q.reshape(Q.shape[0], self.seq_len, self.out_features // 2, 2)
    K = K.reshape(K.shape[0], self.seq_len, self.out_features // 2, 2)


    # Reshape Q, K, V into blocks for each chunk
    Q = Q.reshape(Q.shape[0], n_chunks, self.chunk_size, self.out_features)
    K = K.reshape(K.shape[0], n_chunks, self.chunk_size, self.out_features)
    V = V.reshape(V.shape[0], n_chunks, self.chunk_size, self.out_features)


    # Initialize state
    S = torch.zeros(x.shape[0], self.out_features, self.out_features, device=x.device)
    S.to(x.device)

    out = (Q @ K.transpose(-2, -1) * self.M) @ V
    out.to(x.device)

    for i in range(n_chunks):
      out[:, i, :, :] = out[:, i, :, :] +  Q[:, i, :, :] @ S.transpose(-2, -1)
      S = S + V[:, i, :, :].transpose(-2, -1) @ K[:, i, :, :]

    return out.reshape(x.shape[0], -1, self.out_features)

In [ ]:
class CLAWrapperNextToken(nn.Module):

  def __init__(self, in_features, out_features, chunk_size, vocab_size, seq_len, ffn_multiplier=4, p=0.1):
    super().__init__()
    self.embed = nn.Embedding(vocab_size, in_features)
    self.head = ChunkwiseLinearAttentionHead(in_features, out_features, chunk_size, seq_len)

    d_ff = ffn_multiplier * out_features
    self.ffn = nn.Sequential(
        nn.Linear(out_features, d_ff),
        nn.GELU(),
        nn.Dropout(p),
        nn.Linear(d_ff, out_features),
        nn.Dropout(p),
    )

    self.fc = nn.Linear(out_features, vocab_size)

  def forward(self, x, attention_mask=None):
    embeds = self.embed(x)

    # Absolute position encodings -- use sinusoidal position embeddings a la BERT
    positions = torch.arange(embeds.shape[1], device=embeds.device).unsqueeze(1)
    div_term = torch.exp(torch.arange(0, embeds.shape[2], 2, device=embeds.device) * -(math.log(10000.0) / embeds.shape[2]))
    encodings = torch.zeros_like(embeds)
    encodings[:, :, 0::2] = torch.sin(positions * div_term)
    encodings[:, :, 1::2] = torch.cos(positions * div_term)
    encoded_embeds = embeds + encodings


    attention_vectors = self.head(encoded_embeds)

    # x is B x L x D -- we want output to be B x L x V, so we apply linear layer
    out = self.fc(attention_vectors + self.ffn(attention_vectors))

    # return unnormalized logits for cross entropy loss
    return out

In [ ]:
class CLAWrapperNextToken2(nn.Module):

  def __init__(self, in_features, out_features, chunk_size, vocab_size, seq_len, ffn_multiplier=4, p=0.1):
    super().__init__()
    self.embed = nn.Embedding(vocab_size, in_features)
    self.head1 = ChunkwiseLinearAttentionHead(in_features, out_features, chunk_size, seq_len)

    d_ff = ffn_multiplier * out_features

    self.ffn = nn.Sequential(
        nn.Linear(out_features, d_ff),
        nn.GELU(),
        nn.Dropout(p),
        nn.Linear(d_ff, out_features),
        nn.Dropout(p),
    )

    self.fc_mid = nn.Linear(out_features, in_features)
    self.head2 = ChunkwiseLinearAttentionHead(in_features, out_features, chunk_size, seq_len)

    self.ffn2 = nn.Sequential(
        nn.Linear(out_features, d_ff),
        nn.GELU(),
        nn.Dropout(p),
        nn.Linear(d_ff, out_features),
        nn.Dropout(p),
    )

    self.fc = nn.Linear(out_features, vocab_size)

  def forward(self, x, attention_mask=None):
    embeds = self.embed(x)

    # Absolute position encodings -- use sinusoidal position embeddings a la BERT
    positions = torch.arange(embeds.shape[1], device=embeds.device).unsqueeze(1)
    div_term = torch.exp(torch.arange(0, embeds.shape[2], 2, device=embeds.device) * -(math.log(10000.0) / embeds.shape[2]))
    encodings = torch.zeros_like(embeds)
    encodings[:, :, 0::2] = torch.sin(positions * div_term)
    encodings[:, :, 1::2] = torch.cos(positions * div_term)
    encoded_embeds = embeds + encodings


    attention_vectors1 = self.head1(encoded_embeds)

    # map first attention head outputs to input space of second head, then pass through second head
    next_inps = self.fc_mid(attention_vectors1 + self.ffn(attention_vectors1))
    attention_vectors2 = self.head2(next_inps + encoded_embeds)

    # x is B x L x D -- we want output to be B x L x V, so we apply linear layer
    out = self.fc(attention_vectors2 + self.ffn2(attention_vectors2))

    # return unnormalized logits for cross entropy loss
    return out

In [ ]:
class CLAWrapperTextClassification(nn.Module):

  def __init__(self, in_features, out_features, chunk_size, vocab_size, num_classes, seq_len):
    super().__init__()
    self.embed = nn.Embedding(vocab_size, in_features)
    self.head = ChunkwiseLinearAttentionHead(in_features, out_features, chunk_size, seq_len)
    self.fc = nn.Linear(out_features, num_classes)

  def forward(self, x, attention_mask=None):
    embeds = self.embed(x)
    attention_vectors = self.head(embeds)

    # B-length 1d tensor
    if attention_mask is not None:
      cutoff_indices = attention_mask.sum(dim=1) - 1
    else:
      cutoff_indices = -1

    arange_selector = torch.arange(attention_vectors.shape[0])

    # x is B x L x D -- we want output to be B x C, so we need to remove the L dimension
    out = self.fc(attention_vectors[arange_selector, cutoff_indices, :])

    # return unnormalized logits for cross entropy loss
    return out

## Tests of initial models

In [ ]:
batch_size = 64
seq_len = 1024
dims = 512

data = torch.randn(batch_size, seq_len, dims)
linhead = ChunkwiseLinearAttentionHead(dims, 3*dims, 64, seq_len)
out = linhead(data)
print()
print(data.shape)
print(out.shape)

/tmp/ipython-input-2453875168.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.M = torch.tensor(torch.tril(torch.ones(chunk_size, chunk_size)))


torch.Size([64, 1024, 1536])
torch.Size([64, 1024, 1536])
torch.Size([64, 1024, 1536])

torch.Size([64, 1024, 512])
torch.Size([64, 1024, 1536])


In [ ]:
for i, k in enumerate(linhead.named_modules()):
  print(i)
  print(k)
  print()

0
('', ChunkwiseLinearAttentionHead(
  (Q_linear): Linear(in_features=512, out_features=1536, bias=False)
  (K_linear): Linear(in_features=512, out_features=1536, bias=False)
  (V_linear): Linear(in_features=512, out_features=1536, bias=False)
))

1
('Q_linear', Linear(in_features=512, out_features=1536, bias=False))

2
('K_linear', Linear(in_features=512, out_features=1536, bias=False))

3
('V_linear', Linear(in_features=512, out_features=1536, bias=False))



# AWQ simulation

In [ ]:
# core quantization method (simulated quantization)
def pseudo_quantize_tensor(w, n_bit=4, q_group_size=-1):
    org_w_shape = w.shape
    if q_group_size > 0:
        assert org_w_shape[-1] % q_group_size == 0
        w = w.reshape(-1, q_group_size)

    assert w.dim() == 2

    # Calculate the maximum (\alpha) and minimum values (\beta) in the tensor.
    max_val = w.amax(dim=1, keepdim=True)
    assert max_val.dim() == 2 and max_val.size(0) == w.size(0) and max_val.size(1) == 1
    min_val = w.amin(dim=1, keepdim=True)
    assert min_val.dim() == 2 and min_val.size(0) == w.size(0) and min_val.size(1) == 1

    # Calculate the scale factor and zero point.  (Formula 1 & 2)
    max_int = 2 ** n_bit - 1
    scales = (max_val - min_val).clamp(min=1e-5) / max_int
    assert scales.shape == max_val.shape
    zeros = (-torch.round(min_val / scales)).clamp_(0, max_int)
    assert scales.shape == min_val.shape

    assert torch.isnan(scales).sum() == 0
    assert torch.isnan(w).sum() == 0

    # Quantize W: Map values in the range [\beta, \alpha] to lie within [0, 2^b - 1] (Formula 3)
    w = torch.clamp(torch.round(w / scales) + zeros, 0, max_int)
    assert w.dim() == 2 and w.size(0) == scales.size(0) and w.size(1) == q_group_size

    # Dequantize W (pseudo quantization, the inverse transformation of Formula 3)
    w = (w - zeros) * scales
    assert w.dim() == 2 and w.size(0) == scales.size(0) and w.size(1) == q_group_size

    assert torch.isnan(w).sum() == 0

    w = w.reshape(org_w_shape)
    return w

@torch.no_grad()
def pseudo_quantize_model_weight(
    model, w_bit, q_group_size,
):
    for n, m in model.named_modules():
        if isinstance(m, nn.Linear):
            m.weight.data = pseudo_quantize_tensor(m.weight.data, n_bit=w_bit, q_group_size=q_group_size)

In [1]:
@torch.no_grad()
def get_calib_feat(model, tokenizer, dataset):
    input_dict = dict()
    def stat_input_max_hook(m, x, y, name):
        if isinstance(x, tuple):
            x = x[0]
        x_max = x.view(-1, x.shape[-1]).abs().mean(dim=0).cpu().detach()
        if name not in input_dict:
            input_dict[name] = [x_max]
        else:
            input_dict[name] += [x_max]

    hooks = []
    for name, m in model.named_modules():
        if isinstance(m, nn.Linear):
            hooks.append(
                m.register_forward_hook(
                    partial(stat_input_max_hook, name=name)))

    print("Collecting activation scales...")
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    pbar = tqdm.tqdm(dataset)
    for batch in pbar:
        data_fields = ['input_ids']

        for k in data_fields:
          try:
            batch[k] = torch.stack(batch[k], dim=1)
          except:
            print(batch[k])
            raise ValueError("CLOWN")

        batch = {k: torch.tensor(batch[k]).to(device, non_blocking=True) for k in data_fields}

        model(batch['input_ids'])

    for hook in hooks:
        hook.remove()
    return input_dict

NameError: name 'torch' is not defined

In [ ]:
@torch.no_grad()
def pseudo_quantize_model_salient_weight_fp16(
    model, w_bit, q_group_size, input_feat
):
    for n, m in model.named_modules():
        if isinstance(m, nn.Linear):
            importance = sum(input_feat[n]).float()

            outlier_indices = torch.topk(importance, int(m.weight.data.shape[1]//100)).indices
            assert outlier_indices.dim() == 1

            # Back up the values of the salient weight channels
            outlier = m.weight.data[:, outlier_indices].clone()

            m.weight.data = pseudo_quantize_tensor(m.weight.data, n_bit=w_bit, q_group_size=q_group_size)

            m.weight.data[:, outlier_indices] = outlier

In [ ]:
pseudo_quantize_model_weight(linhead, w_bit=3, q_group_size=128)

# Evaluate the model
out_quant_pure = linhead(data)
print((out_quant_pure - out).abs().sum())

KeyboardInterrupt: 

In [ ]:
print(((out_quant_pure - out).abs().sum())/(out_quant_pure.numel()))

In [ ]:
out.mean(axis=None)

# Load Data for Tasks

There are datasets we will be considering in this notebook.

1. Penn Tree Bank -- this is a next-token prediction task used by early variants of GPT.

2. IMDB database -- this is a binary text classification task, sorting movie reviews into positive/negative sentiment.

## Penn Treebank

We will consider two variants with Penn Treebank. We can either feed each sentence as an independent input, or we can replace end of sentences with an EOS token, and divide the input into strings of fixed length.

We will build data pipelines for both functionalities here, and experiment with them as if they were separate tasks. In some ways, the Penn Treebank task may have a part (a) and a part (b).

The components of this module are as follows:
1. Vocab: maps tokens to ints and ints to tokens. Converts unseen tokens into `<unk>`.
2. Tokenizer: uses the vocab object to take raw text files and convert them into token streams.
3. Dataloaders: loads train/val/test data for torch models. slightly different behavior necessary for training than for val and test (sliding instead of chunking for context creation).

In [ ]:
!unzip /content/ptb-copy-paste.zip

Archive:  /content/ptb-copy-paste.zip
   creating: ptb-copy-paste/
  inflating: ptb-copy-paste/train.txt  
  inflating: __MACOSX/ptb-copy-paste/._train.txt  
  inflating: ptb-copy-paste/test.txt  
  inflating: __MACOSX/ptb-copy-paste/._test.txt  
  inflating: ptb-copy-paste/val.txt  
  inflating: __MACOSX/ptb-copy-paste/._val.txt  


In [ ]:
read_dirs = ['/content/ptb-copy-paste/train.txt', '/content/ptb-copy-paste/val.txt']

### Vocab

In [ ]:
class Vocab:
    """
    Simple word-level vocabulary.

    - Built from token streams (train only).
    - Provides stoi/itos mappings with reserved specials.
    - OOV tokens map to <unk>.

    Args:
        min_freq: keep tokens with frequency >= min_freq
        max_size: keep at most this many tokens (excluding specials); ties broken by freq then lexicographic
        specials: list of reserved tokens placed at the start of vocab
    """
    def __init__(self, min_freq, max_size, specials):
        self.min_freq = int(min_freq)
        self.max_size = max_size if (max_size is None) else int(max_size)
        self.specials = specials or ["<unk>", "<eos>"]
        # mappings
        self.stoi = {} # Dict[str, int]
        self.itos = [] # List [str]
        self._frozen = False

    @property
    def unk_token(self) -> str:
        return self.specials[0]

    @property
    def eos_token(self) -> str:
        # we include <eos> as a special for convenience; you may not need an id for it explicitly
        return self.specials[1] if len(self.specials) > 1 else "<eos>"


    def file_tokens(self, file_list):
      for f in file_list:
        with open(f, "r", encoding="utf-8") as f:
          for raw in f:
            s = raw.split()
            s.append('<eos>')
            yield s


    def build(self, file_list=read_dirs):
        """
        Build vocabulary from an iterable of tokens (typically the entire train set).
        """
        if self._frozen:
            raise RuntimeError("Vocab is frozen; cannot rebuild.")

        for token_stream in self.file_tokens(file_list=file_list):
          for token in token_stream:
            if token not in self.stoi and token not in self.specials:
              self.stoi[token] = len(self.itos)
              self.itos.append(token)

        for special in self.specials:
          if special not in self.stoi:
            self.stoi[special] = len(self.itos)
            self.itos.append(special)


        '''
        counter = Counter(tokens)
        # Filter by min_freq
        items = [(tok, freq) for tok, freq in counter.items() if freq >= self.min_freq]
        # Sort: frequency (desc) then token (asc) for determinism
        items.sort(key=lambda x: (-x[1], x[0]))

        if self.max_size is not None:
            items = items[: self.max_size]

        # Start with specials
        self.itos = list(dict.fromkeys(self.specials))  # preserve order, dedupe if passed duplicates
        # Then add items not already in specials
        for tok, _ in items:
            if tok not in self.itos:
                self.itos.append(tok)

        self.stoi = {tok: i for i, tok in enumerate(self.itos)}
        '''
        self._frozen = True

    def __len__(self):
        return len(self.itos)

    def to_id(self, tok: str):
        """
        Map token -> id; unknowns -> <unk>.
        """
        if tok in self.stoi:
            return self.stoi[tok]
        return self.stoi[self.unk_token]

    def to_token(self, idx: int):
        return self.itos[idx]

    def ids(self, tokens):
        return [self.to_id(t) for t in tokens]

In [ ]:
v = Vocab(min_freq=0, max_size=None, specials=None)
v.specials
v.build()

'preliminary'

In [ ]:
# Save Vocab object
import pickle
with open('/content/ptb-vocab-word.pkl', 'wb') as f:
  pickle.dump(v, f)

In [ ]:
# Test load
import pickle
with open('/content/ptb-vocab-word.pkl', 'rb') as f:
  v2 = pickle.load(f)
  print(v2.specials)
  print(len(v2))
  print(v2.ids(['the', 'fat', 'rat', 'was', 'eaten', 'by', 'the', 'cat', 'quickly']))

['<unk>', '<eos>']
10000
[30, 1795, 9998, 52, 9998, 227, 30, 3108, 1843]


In [ ]:
len(v2)

10000

### Tokenizer



In [ ]:
# use v2.ids() for tokenization

### Preprocessing

In [ ]:
ptb_root_dir = '/content/ptb-copy-paste/'
ptb_write_dir = '/content/ptb-copy-paste-tokenized/'

In [ ]:
def load_ptb_help(filename):
  final_array = np.array([], dtype=np.int64)
  with open(filename, 'r') as f:
    for raw in f:
      s = raw.split()
      s.append('<eos>')
      final_array = np.concatenate([final_array, np.array(v2.ids(s))])

  return final_array

def load_ptb_train():
  return load_ptb_help(ptb_root_dir + 'train.txt')

def load_ptb_val():
  return load_ptb_help(ptb_root_dir + 'val.txt')

def load_ptb_test():
  return load_ptb_help(ptb_root_dir + 'test.txt')

def load_ptb_all():
  return {'train': load_ptb_train(), 'val': load_ptb_val(), 'test': load_ptb_test()}

In [ ]:
all_ptb_tokens = load_ptb_all()
for k in all_ptb_tokens:
  print(k, len(all_ptb_tokens[k]))

train 929589
val 73760
test 82430


### Dataloader

We need to convert a large corpus of tokens into a dataset, and then create a dataset dict using the datasets.

Thus, the functionalities we need to build are as folloows:
1. Convert corpus into dictionary of fields
2. Convert fields into `Dataset`
3. Aggregate `Dataset` objects into `DatasetDict`.

In [ ]:
def corpus_to_fields(corpus, seq_len, stride=None):
  if stride == None:
    stride = seq_len

  if len(corpus) % seq_len != 1:
    corpus = corpus[:-(len(corpus) % seq_len)+1]

  start_indices = np.arange(0, len(corpus) - seq_len, stride)

  input_ids = np.array([corpus[i:i+seq_len] for i in start_indices])
  labels = np.array([corpus[i+1:i+1+seq_len] for i in start_indices])

  attention_mask = np.ones_like(input_ids)

  return {'input_ids': input_ids, 'labels': labels, 'attention_mask': attention_mask}

In [ ]:
for k in all_ptb_tokens:
  stride = 32 if k == 'train' else 1
  all_ptb_tokens[k] = Dataset.from_dict(corpus_to_fields(all_ptb_tokens[k], seq_len=32, stride=stride))

all_ptb_tokens = DatasetDict(all_ptb_tokens)

In [ ]:
all_ptb_tokens.save_to_disk(ptb_write_dir)
print(f"Saved packed dataset to: {ptb_write_dir}")
print(all_ptb_tokens)

Saving the dataset (0/1 shards):   0%|          | 0/29049 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards): 0 examples [00:00, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/82369 [00:00<?, ? examples/s]

Saved packed dataset to: /content/ptb-copy-paste-tokenized/
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels', 'attention_mask'],
        num_rows: 29049
    })
    val: Dataset({
        features: ['input_ids', 'labels', 'attention_mask'],
        num_rows: 0
    })
    test: Dataset({
        features: ['input_ids', 'labels', 'attention_mask'],
        num_rows: 82369
    })
})


In [ ]:
!zip -r /content/ptb_copy_paste_tokenized.zip /content/ptb-copy-paste-tokenized

updating: content/ptb-copy-paste-tokenized/ (stored 0%)
updating: content/ptb-copy-paste-tokenized/dataset_dict.json (deflated 3%)
updating: content/ptb-copy-paste-tokenized/test/ (stored 0%)
updating: content/ptb-copy-paste-tokenized/test/dataset_info.json (deflated 69%)
updating: content/ptb-copy-paste-tokenized/test/state.json (deflated 38%)
updating: content/ptb-copy-paste-tokenized/test/data-00001-of-00002.arrow (deflated 99%)
updating: content/ptb-copy-paste-tokenized/test/data-00000-of-00002.arrow (deflated 99%)
updating: content/ptb-copy-paste-tokenized/val/ (stored 0%)
updating: content/ptb-copy-paste-tokenized/val/dataset_info.json (deflated 69%)
updating: content/ptb-copy-paste-tokenized/val/data-00000-of-00001.arrow (deflated 99%)
updating: content/ptb-copy-paste-tokenized/val/state.json (deflated 39%)
updating: content/ptb-copy-paste-tokenized/train/ (stored 0%)
updating: content/ptb-copy-paste-tokenized/train/dataset_info.json (deflated 69%)
updating: content/ptb-copy-pas

In [ ]:
!unzip /content/ptb_copy_paste_tokenized.zip
!rm -rf /content/ptb-copy-paste-tokenized
!mv -f /content/content/ptb-copy-paste-tokenized/ /content
!rm -rf /content/content

!ls

Archive:  /content/ptb_copy_paste_tokenized.zip
   creating: content/ptb-copy-paste-tokenized/
  inflating: content/ptb-copy-paste-tokenized/dataset_dict.json  
   creating: content/ptb-copy-paste-tokenized/test/
  inflating: content/ptb-copy-paste-tokenized/test/dataset_info.json  
  inflating: content/ptb-copy-paste-tokenized/test/state.json  
  inflating: content/ptb-copy-paste-tokenized/test/data-00001-of-00002.arrow  
  inflating: content/ptb-copy-paste-tokenized/test/data-00000-of-00002.arrow  
   creating: content/ptb-copy-paste-tokenized/val/
  inflating: content/ptb-copy-paste-tokenized/val/dataset_info.json  
  inflating: content/ptb-copy-paste-tokenized/val/data-00000-of-00001.arrow  
  inflating: content/ptb-copy-paste-tokenized/val/state.json  
   creating: content/ptb-copy-paste-tokenized/train/
  inflating: content/ptb-copy-paste-tokenized/train/dataset_info.json  
  inflating: content/ptb-copy-paste-tokenized/train/data-00000-of-00001.arrow  
  inflating: content/ptb-co

In [ ]:
all_ptb_tokens = DatasetDict.load_from_disk(ptb_write_dir)

In [ ]:
all_ptb_tokens

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels', 'attention_mask'],
        num_rows: 29049
    })
    val: Dataset({
        features: ['input_ids', 'labels', 'attention_mask'],
        num_rows: 0
    })
    test: Dataset({
        features: ['input_ids', 'labels', 'attention_mask'],
        num_rows: 82369
    })
})

In [ ]:
batch_size = 128
ptb_train_loader = DataLoader(all_ptb_tokens["train"], batch_size=batch_size, shuffle=True)
# ptb_val_loader = DataLoader(all_ptb_tokens["val"], batch_size=batch_size, shuffle=True)
ptb_test_loader = DataLoader(all_ptb_tokens["test"], batch_size=batch_size, shuffle=True)

## IMDB Reviews

In [ ]:
!unzip imdb-dataset-classic.zip

Streaming output truncated to the last 5000 lines.
  inflating: imdb_dataset_classic /train/pos/9260_7.txt  
  inflating: __MACOSX/imdb_dataset_classic /train/pos/._9260_7.txt  
  inflating: imdb_dataset_classic /train/pos/1599_7.txt  
  inflating: __MACOSX/imdb_dataset_classic /train/pos/._1599_7.txt  
  inflating: imdb_dataset_classic /train/pos/2174_8.txt  
  inflating: __MACOSX/imdb_dataset_classic /train/pos/._2174_8.txt  
  inflating: imdb_dataset_classic /train/pos/2309_9.txt  
  inflating: __MACOSX/imdb_dataset_classic /train/pos/._2309_9.txt  
  inflating: imdb_dataset_classic /train/pos/12034_10.txt  
  inflating: __MACOSX/imdb_dataset_classic /train/pos/._12034_10.txt  
  inflating: imdb_dataset_classic /train/pos/11703_9.txt  
  inflating: __MACOSX/imdb_dataset_classic /train/pos/._11703_9.txt  
  inflating: imdb_dataset_classic /train/pos/5619_9.txt  
  inflating: __MACOSX/imdb_dataset_classic /train/pos/._5619_9.txt  
  inflating: imdb_dataset_classic /train/pos/2928_10.t

### Tokenizer

For the IMDB review dataset, we will be tokenizing by using the BERT tokenizer.

In [ ]:
imdb_tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
sample_sentence = "Like an alien, I am sent here to destroy you. And there's a million others just like me, who walk, talk, and act like me."
vv = imdb_tokenizer(sample_sentence)
vv

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

{'input_ids': [101, 2409, 1126, 8143, 117, 146, 1821, 1850, 1303, 1106, 5535, 1128, 119, 1262, 1175, 112, 188, 170, 1550, 1639, 1198, 1176, 1143, 117, 1150, 2647, 117, 2037, 117, 1105, 2496, 1176, 1143, 119, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
imdb_tokenizer

BertTokenizerFast(name_or_path='bert-base-cased', vocab_size=28996, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

In [ ]:
len(imdb_tokenizer)

28996

In [ ]:
# Check for longest review in train/neg, train/pos, test/neg, and test/pos

imdb_root_dir = '/content/imdb_dataset_classic/'
os.listdir(imdb_root_dir)
dirs_to_consider = ['train/neg', 'train/pos', 'test/neg', 'test/pos']
for d in dirs_to_consider:
  max_len = 0
  for f in os.listdir(imdb_root_dir + d):
    with open(imdb_root_dir + d + '/' + f, 'r') as f:
      max_len = max(max_len, len(f.read()))
  print(d, max_len)

train/neg 8969
train/pos 13704
test/neg 6385
test/pos 12988


In [ ]:
dirs_to_consider

['train/neg', 'train/pos', 'test/neg', 'test/pos']

### Preprocessing

Here, we will use the tokenizer we created to tokenize all the data in `imdb_dataset_classic`. We will then stored the tokenized copies as PyArrow datasets, for future use in training.

In [ ]:
imdb_root_dir = '/content/imdb_dataset_classic/'
imdb_write_dir = '/content/imdb_dataset_classic_tokenized/'
dirs_to_consider = ['train/neg', 'train/pos', 'test/neg', 'test/pos']

We load the PyArrow datasets, which consist of tokenized input streams and labels. We attach these datasets to dataloaders, which will be used for training and evaluation loops later on.

In [ ]:
# load files
def load_files(split='all'):
  for d in dirs_to_consider:
    if split != 'all' and not d.startswith(split):
      print(d)
      continue
    cur_directory = imdb_root_dir + d
    if cur_directory.endswith('neg'):
      label = 0
    else:
      label = 1
    for p in Path(cur_directory).glob('*.txt'):
      plaintext = Path(p).read_text(encoding="utf-8", errors="ignore")
      yield {'text': plaintext, 'label': label}

def load_files_train():
  return load_files(split='train')

def load_files_test():
  return load_files(split='test')


raw_dataset = Dataset.from_generator(load_files)
train_dataset = Dataset.from_generator(load_files_train)
test_dataset = Dataset.from_generator(load_files_test)

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

test/neg
test/pos


Generating train split: 0 examples [00:00, ? examples/s]

train/neg
train/pos


In [ ]:
raw_dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 50000
})

In [ ]:
train_dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 25000
})

In [ ]:
test_dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 25000
})

In [ ]:
test_dataset[0]

{'text': 'I carefully checked if there\'s another movie named as this one, and there isn\'t ! But I really don\'t think we all saw the same movie ! There\'s no way ! How can you vote more than "1" for this movie ?! The idea of this movie let\'s say it\'s acceptable. Oh, and the acting of Dan Gordon (Chris) is quite good. But those are the only two things acceptable in this project. The others are... awful ? It\'s a very delicate word to describe the acting of the other actors, the directing, the (so said) "special" effects, even the way that the crew was filming ! I don\'t even like the way that the camera operators were moving to record the scenes ! This may be the most miserable film I\'ve ever seen. I really don\'t remember a movie lower than this one... Maybe there is, but... I don\'t think so... Ehh, what\'s done, it\'s done... That\'s the movie and there\'s too late for anyone to change anything. I\'ve voted "1", but my realistic vote starts with a "-" (minus) in front....',
 'la

In [ ]:
imdb_ds_dict = DatasetDict({"train": train_dataset, "test": test_dataset})

In [ ]:
def imdb_tokenize(batch):
  return imdb_tokenizer(batch["text"], truncation=True, padding='max_length', max_length=512)

imdb_ds_dict_tokenized = imdb_ds_dict.map(imdb_tokenize, batched=True)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

In [ ]:
imdb_ds_dict_tokenized.keys()

dict_keys(['train', 'test'])

In [ ]:
imdb_ds_dict_tokenized["train"].features

{'text': Value('string'),
 'label': Value('int64'),
 'input_ids': List(Value('int32')),
 'token_type_ids': List(Value('int8')),
 'attention_mask': List(Value('int8'))}

In [ ]:
imdb_ds_dict_tokenized.save_to_disk(imdb_write_dir)
print(f"Saved packed dataset to: {imdb_write_dir}")
print(imdb_ds_dict_tokenized)

Saving the dataset (0/1 shards):   0%|          | 0/25000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/25000 [00:00<?, ? examples/s]

Saved packed dataset to: /content/imdb_dataset_classic_tokenized/
DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 25000
    })
})


In [ ]:
from google.colab import files
files.download('/content/imdb_dataset_classic_tokenized')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!zip -r /content/imdb_dataset_classic_tokenized.zip /content/imdb_dataset_classic_tokenized

  adding: content/imdb_dataset_classic_tokenized/ (stored 0%)
  adding: content/imdb_dataset_classic_tokenized/test/ (stored 0%)
  adding: content/imdb_dataset_classic_tokenized/test/dataset_info.json (deflated 69%)
  adding: content/imdb_dataset_classic_tokenized/test/state.json (deflated 38%)
  adding: content/imdb_dataset_classic_tokenized/test/data-00000-of-00001.arrow (deflated 77%)
  adding: content/imdb_dataset_classic_tokenized/train/ (stored 0%)
  adding: content/imdb_dataset_classic_tokenized/train/dataset_info.json (deflated 69%)
  adding: content/imdb_dataset_classic_tokenized/train/state.json (deflated 37%)
  adding: content/imdb_dataset_classic_tokenized/train/data-00000-of-00001.arrow (deflated 77%)
  adding: content/imdb_dataset_classic_tokenized/dataset_dict.json (stored 0%)


In [ ]:
!unzip /content/imdb_dataset_classic_tokenized.zip
!rm -rf /content/imdb_dataset_classic_tokenized
!mv -f /content/content/imdb_dataset_classic_tokenized/ /content
!rm -rf /content/content

!ls

Archive:  /content/imdb_dataset_classic_tokenized.zip
   creating: content/imdb_dataset_classic_tokenized/
   creating: content/imdb_dataset_classic_tokenized/test/
  inflating: content/imdb_dataset_classic_tokenized/test/dataset_info.json  
  inflating: content/imdb_dataset_classic_tokenized/test/state.json  
  inflating: content/imdb_dataset_classic_tokenized/test/data-00000-of-00001.arrow  
   creating: content/imdb_dataset_classic_tokenized/train/
  inflating: content/imdb_dataset_classic_tokenized/train/dataset_info.json  
  inflating: content/imdb_dataset_classic_tokenized/train/state.json  
  inflating: content/imdb_dataset_classic_tokenized/train/data-00000-of-00001.arrow  
 extracting: content/imdb_dataset_classic_tokenized/dataset_dict.json  
imdb_dataset_classic_tokenized	imdb_dataset_classic_tokenized.zip  sample_data


In [ ]:
imdb_ds_dict_tokenized = DatasetDict.load_from_disk(imdb_write_dir)

### Dataloaders

In [ ]:
batch_size = 64

imdb_train_loader = DataLoader(imdb_ds_dict_tokenized["train"], batch_size=batch_size, shuffle=True)
imdb_test_loader = DataLoader(imdb_ds_dict_tokenized["test"], batch_size=batch_size, shuffle=True)

# Metrics

We need different metrics for each of the tasks.

1. Penn Tree Bank -- this is next-token prediction. Therefore, we need to be able to compute multiclass (N = vocab size) log-loss of token prediction for training and perplexity for eval.

2. IMDB -- this is binary prediction. Therefore, we need binary log-loss for training, as well as accuracy, F1 score, precision, recall on different thresholds for eval. AUC functionality would be ideal here too.

## Penn Tree Bank

In [ ]:
class PennTreeBankMetrics:
  # will be entirely static methods -- no need for constructor

  @staticmethod
  def loss(y_pred, y_true):
    # y_pred is B x L x V, and y_true is B x L

    y_pred = y_pred.view(-1, y_pred.shape[-1])
    y_true = y_true.view(-1)

    return F.cross_entropy(y_pred, y_true)

  @staticmethod
  def perplexity(y_pred, y_true):
    return torch.exp(PennTreeBankMetrics.loss(y_pred, y_true))

In [ ]:
torch.randn(3, 5)

tensor([[ 0.2054, -1.5499, -0.4270, -0.4302,  0.3877],
        [ 1.2357,  0.3781,  0.6738, -1.7964, -0.0053],
        [ 1.8661, -0.3323, -0.9979,  1.0900, -0.2780]])

In [ ]:
torch.randint(5, (3,), dtype=torch.int64)

tensor([0, 1, 4])

## IMDB

In [ ]:
class IMDBMetrics:
  # Will be entirely static methods -- no need for constructor

  @staticmethod
  def loss(y_pred, y_true):
    # y_pred is B x 2 -- we submit as is with CEL
    return F.cross_entropy(y_pred, y_true)

  @staticmethod
  def accuracy(y_pred, y_true):
    return (y_pred.argmax(dim=1) == y_true).float().mean()

  @staticmethod
  def precision(y_pred, y_true):
    ps = []
    thresh = np.arange(0, 1, 0.05)
    for t in thresh:
      true_pos = (y_pred[:, 1] > t) & (y_true == 1)
      false_pos = (y_pred[:, 1] > t) & (y_true == 0)
      precision = true_pos.sum() / (true_pos.sum() + false_pos.sum())
      ps.append(precision)
    return np.array(ps)

  @staticmethod
  def recall(y_pred, y_true):
    thresh = np.arange(0, 1, 0.05)
    rs = []
    for t in thresh:
      true_pos = (y_pred[:, 1] > t) & (y_true == 1)
      false_neg = (y_pred[:, 1] < t) & (y_true == 1)
      recall = true_pos.sum() / (true_pos.sum() + false_neg.sum())
      rs.append(recall)
    return np.array(rs)

  @staticmethod
  def f1(y_pred, y_true):
    ps = IMDBMetrics.precision(y_pred, y_true)
    rs = IMDBMetrics.recall(y_pred, y_true)
    return 2 * ps * rs / (ps + rs)

  @staticmethod
  def auc(y_pred, y_true):
    ps = IMDBMetrics.precision(y_pred, y_true)
    rs = IMDBMetrics.recall(y_pred, y_true)
    return np.trapz(y=ps, x=rs)

# Train + Val Loop

## Penn Treebank

In [ ]:
model = CLAWrapperNextToken(in_features=64, out_features=64, chunk_size=16, vocab_size=len(v2), seq_len=32)
model

/tmp/ipython-input-769934777.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  M = torch.tensor(torch.tril(torch.ones(chunk_size, chunk_size)))


CLAWrapperNextToken(
  (embed): Embedding(10000, 64)
  (head): ChunkwiseLinearAttentionHead(
    (Q_linear): Linear(in_features=64, out_features=64, bias=False)
    (K_linear): Linear(in_features=64, out_features=64, bias=False)
    (V_linear): Linear(in_features=64, out_features=64, bias=False)
  )
  (ffn): Sequential(
    (0): Linear(in_features=64, out_features=256, bias=True)
    (1): GELU(approximate='none')
    (2): Dropout(p=0.1, inplace=False)
    (3): Linear(in_features=256, out_features=64, bias=True)
    (4): Dropout(p=0.1, inplace=False)
  )
  (fc): Linear(in_features=64, out_features=10000, bias=True)
)

In [ ]:
model = CLAWrapperNextToken2(in_features=32, out_features=32, chunk_size=16, vocab_size=len(v2), seq_len=32)

/tmp/ipython-input-769934777.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  M = torch.tensor(torch.tril(torch.ones(chunk_size, chunk_size)))


In [ ]:
epochs = 500
lr = 5e-4
betas = (0.9, 0.999)
weight_decay = 0.01

optimizer = torch.optim.Adam(model.parameters(), lr=lr)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr, betas=betas, weight_decay=weight_decay)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)

for epoch in tqdm.tqdm(range(epochs)):
  total_loss = 0.0
  for step, batch in tqdm.tqdm(enumerate(ptb_train_loader)):
    model.train()
    data_fields = ['input_ids', 'labels']

    for k in data_fields:
      try:
        batch[k] = torch.stack(batch[k], dim=1)
      except:
        print(batch[k])
        raise ValueError("CLOWN")

    batch = {k: torch.tensor(batch[k]).to(device, non_blocking=True) for k in data_fields}

    y_pred = model(batch['input_ids'])
    loss = PennTreeBankMetrics.loss(y_pred, batch['labels'])

    total_loss += loss.item()

    loss.backward()
    optimizer.step()
    optimizer.zero_grad(set_to_none=True)

    # Periodic loss updates on training
    if (step + 1) % 100 == 0:
      avg_train_loss = total_loss / (step + 1)
      print(f"Epoch: {epoch + 1}, Step: {step + 1}, Loss: {avg_train_loss}")

  if epoch % 5 != 4:
    continue

  model.eval()
  with torch.no_grad():
    losses = []
    perplexities = []
    for step, batch in tqdm.tqdm(enumerate(ptb_test_loader)):
      data_fields = ['input_ids', 'labels']

      for k in data_fields:
        try:
          batch[k] = torch.stack(batch[k], dim=1)
        except:
          print(batch[k])
          raise ValueError("CLOWN")


      batch = {k: torch.tensor(batch[k]).to(device, non_blocking=True) for k in data_fields}
      y_pred = model(batch['input_ids'])

      loss = PennTreeBankMetrics.loss(y_pred, batch['labels'])
      losses.append(loss.item())

      ppl = PennTreeBankMetrics.perplexity(y_pred, batch['labels'])
      perplexities.append(ppl.item())

    avg_test_loss = np.mean(losses)
    avg_test_ppl = np.mean(perplexities)
    print()
    print(f"Epoch: {epoch + 1}, Test Loss: {avg_test_loss}, Test Perplexity: {avg_test_ppl}")
    print()

  0%|          | 0/500 [00:00<?, ?it/s]
0it [00:00, ?it/s]/tmp/ipython-input-1909653391.py:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch = {k: torch.tensor(batch[k]).to(device, non_blocking=True) for k in data_fields}

5it [00:00, 46.15it/s]
10it [00:00, 47.35it/s]
15it [00:00, 47.38it/s]
20it [00:00, 47.69it/s]
25it [00:00, 47.91it/s]
30it [00:00, 48.05it/s]
35it [00:00, 48.11it/s]
40it [00:00, 48.08it/s]
45it [00:00, 48.25it/s]
50it [00:01, 48.43it/s]
55it [00:01, 48.58it/s]
60it [00:01, 48.65it/s]
65it [00:01, 48.63it/s]
70it [00:01, 48.62it/s]
75it [00:01, 48.48it/s]
80it [00:01, 48.42it/s]
85it [00:01, 48.36it/s]
90it [00:01, 48.43it/s]
95it [00:01, 48.49it/s]
100it [00:02, 48.53it/s]
105it [00:02, 48.64it/s]

Epoch: 1, Step: 100, Loss: 4.949657382965088



110it [00:02, 48.68it/s]
115it [00:02, 48.71it/s]
120it [00:02, 48.77it/s]
125it [00:02, 48.75it/s]
130it [00:02, 48.78it/s]
135it [00:02, 48.81it/s]
140it [00:02, 48.75it/s]
145it [00:02, 48.75it/s]
150it [00:03, 48.72it/s]
155it [00:03, 48.83it/s]
160it [00:03, 48.91it/s]
165it [00:03, 48.98it/s]
170it [00:03, 49.00it/s]
175it [00:03, 49.02it/s]
180it [00:03, 48.90it/s]
185it [00:03, 48.92it/s]
190it [00:03, 48.61it/s]
195it [00:04, 48.73it/s]
200it [00:04, 48.66it/s]
205it [00:04, 48.52it/s]

Epoch: 1, Step: 200, Loss: 4.963284723758697



210it [00:04, 48.54it/s]
215it [00:04, 47.98it/s]
220it [00:04, 48.10it/s]
227it [00:04, 48.46it/s]
  0%|          | 1/500 [00:04<38:58,  4.69s/it]
0it [00:00, ?it/s]
5it [00:00, 48.11it/s]
10it [00:00, 47.94it/s]
15it [00:00, 48.27it/s]
20it [00:00, 45.77it/s]
25it [00:00, 46.80it/s]
30it [00:00, 46.83it/s]
35it [00:00, 46.65it/s]
40it [00:00, 47.04it/s]
45it [00:00, 47.15it/s]
50it [00:01, 47.34it/s]
55it [00:01, 47.77it/s]
60it [00:01, 48.05it/s]
65it [00:01, 48.32it/s]
70it [00:01, 48.51it/s]
75it [00:01, 48.67it/s]
80it [00:01, 48.75it/s]
85it [00:01, 48.87it/s]
90it [00:01, 48.84it/s]
95it [00:01, 48.92it/s]
100it [00:02, 48.88it/s]
105it [00:02, 48.89it/s]

...

Epoch: 500, Step: 100, Loss: 4.452858281135559



110it [00:02, 44.03it/s]
115it [00:02, 45.03it/s]
120it [00:02, 45.84it/s]
125it [00:02, 45.92it/s]
130it [00:02, 46.62it/s]
135it [00:02, 46.29it/s]
140it [00:03, 47.01it/s]
145it [00:03, 47.26it/s]
150it [00:03, 47.55it/s]
155it [00:03, 47.49it/s]
160it [00:03, 47.52it/s]
165it [00:03, 47.68it/s]
170it [00:03, 45.65it/s]
175it [00:03, 46.05it/s]
180it [00:03, 44.71it/s]
185it [00:04, 43.13it/s]
190it [00:04, 44.22it/s]
195it [00:04, 45.26it/s]
200it [00:04, 44.91it/s]
205it [00:04, 44.60it/s]

Epoch: 500, Step: 200, Loss: 4.47013290643692



210it [00:04, 43.40it/s]
215it [00:04, 44.61it/s]
220it [00:04, 45.23it/s]
227it [00:04, 46.06it/s]

0it [00:00, ?it/s]
7it [00:00, 60.68it/s]
14it [00:00, 61.33it/s]
21it [00:00, 60.29it/s]
28it [00:00, 58.23it/s]
35it [00:00, 59.81it/s]
42it [00:00, 60.10it/s]
49it [00:00, 58.35it/s]
56it [00:00, 59.68it/s]
63it [00:01, 60.64it/s]
70it [00:01, 61.35it/s]
77it [00:01, 61.86it/s]
84it [00:01, 60.67it/s]
91it [00:01, 59.14it/s]
98it [00:01, 60.27it/s]
105it [00:01, 61.08it/s]
112it [00:01, 61.32it/s]
119it [00:01, 61.56it/s]
126it [00:02, 59.26it/s]
132it [00:02, 58.08it/s]
139it [00:02, 59.68it/s]
146it [00:02, 60.76it/s]
153it [00:02, 61.15it/s]
160it [00:02, 58.93it/s]
167it [00:02, 59.74it/s]
174it [00:02, 60.64it/s]
181it [00:03, 61.31it/s]
188it [00:03, 61.61it/s]
195it [00:03, 61.91it/s]
202it [00:03, 62.28it/s]
209it [00:03, 62.51it/s]
216it [00:03, 62.69it/s]
223it [00:03, 62.54it/s]
230it [00:03, 62.53it/s]
237it [00:03, 62.67it/s]
244it [00:04, 62.82it/s]
251it [00:04, 62.75


Epoch: 500, Test Loss: 5.583091965373258, Test Perplexity: 267.0334675237999



## IMDB

In [ ]:
model = CLAWrapperTextClassification(in_features=512, out_features=512, chunk_size=64, vocab_size=28996, num_classes=2, seq_len=512)

/tmp/ipython-input-769934777.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  M = torch.tensor(torch.tril(torch.ones(chunk_size, chunk_size)))


In [ ]:
epochs = 10
lr = 1e-3
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)

for epoch in tqdm.tqdm(range(epochs)):
  total_loss = 0.0
  for step, batch in tqdm.tqdm(enumerate(imdb_train_loader)):
    model.train()
    data_fields = ['input_ids', 'attention_mask', 'label']

    for k in data_fields:
      if k == 'label':
        continue
      try:
        batch[k] = torch.stack(batch[k], dim=1)
      except:
        print(batch[k])
        raise ValueError("CLOWN")

    batch = {k: torch.tensor(batch[k]).to(device, non_blocking=True) for k in data_fields}

    y_pred = model(batch['input_ids'], attention_mask=batch['attention_mask'])
    loss = IMDBMetrics.loss(y_pred, batch['label'])

    total_loss += loss.item()

    loss.backward()
    optimizer.step()
    optimizer.zero_grad(set_to_none=True)

    # Periodic loss updates on training
    if (step + 1) % 100 == 0:
      avg_train_loss = total_loss / (step + 1)
      print(f"Epoch: {epoch + 1}, Step: {step + 1}, Loss: {avg_train_loss}")


  accuracies = []
  losses = []
  model.eval()
  with torch.no_grad():
    for step, batch in tqdm.tqdm(enumerate(imdb_test_loader)):
      data_fields = ['input_ids', 'attention_mask', 'label']

      for k in data_fields:
        if k == 'label':
          continue
        try:
          batch[k] = torch.stack(batch[k], dim=1)
        except:
          print(batch[k])
          raise ValueError("CLOWN")


      batch = {k: torch.tensor(batch[k]).to(device, non_blocking=True) for k in data_fields}
      y_pred = model(batch['input_ids'], attention_mask=batch['attention_mask'])

      loss = IMDBMetrics.loss(y_pred, batch['label'])
      accuracies.append(IMDBMetrics.accuracy(y_pred, batch['label']).item())
      losses.append(loss.item())


    avg_loss = np.mean(losses)
    avg_accuracy = np.mean(accuracies)
    print()
    print(f"Epoch: {epoch + 1}, Test Loss: {avg_loss}, Test Accuracy: {avg_accuracy}")
    print()

  0%|          | 0/30 [00:00<?, ?it/s]
0it [00:00, ?it/s]/tmp/ipython-input-2536824166.py:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch = {k: torch.tensor(batch[k]).to(device, non_blocking=True) for k in data_fields}

1it [00:00,  9.44it/s]
2it [00:00,  8.58it/s]
3it [00:00,  8.26it/s]
4it [00:00,  8.25it/s]
5it [00:00,  8.19it/s]
6it [00:00,  8.15it/s]
7it [00:00,  8.06it/s]
8it [00:00,  8.12it/s]
9it [00:01,  8.11it/s]
10it [00:01,  8.09it/s]
11it [00:01,  8.09it/s]
12it [00:01,  8.08it/s]
13it [00:01,  8.07it/s]
14it [00:01,  8.09it/s]
15it [00:01,  8.07it/s]
16it [00:01,  8.07it/s]
17it [00:02,  8.06it/s]
18it [00:02,  8.07it/s]
19it [00:02,  8.06it/s]
20it [00:02,  8.06it/s]
21it [00:02,  8.06it/s]
22it [00:02,  8.06it/s]
23it [00:02,  8.05it/s]
24it [00:02,  8.05it/s]
25it [00:03,  8.06it/s]
26it [00:03,  8.05it/s]
27it 

Epoch: 1, Step: 100, Loss: 73.28202268183232



102it [00:12,  8.05it/s]
103it [00:12,  8.06it/s]
104it [00:12,  8.05it/s]
105it [00:13,  7.99it/s]
106it [00:13,  8.06it/s]
107it [00:13,  8.04it/s]
108it [00:13,  8.05it/s]
109it [00:13,  8.05it/s]
110it [00:13,  7.99it/s]
111it [00:13,  8.06it/s]
112it [00:13,  8.05it/s]
113it [00:14,  8.05it/s]
114it [00:14,  8.04it/s]
115it [00:14,  8.04it/s]
116it [00:14,  8.04it/s]
117it [00:14,  8.00it/s]
118it [00:14,  8.06it/s]
119it [00:14,  8.07it/s]
120it [00:14,  8.05it/s]
121it [00:15,  8.06it/s]
122it [00:15,  8.05it/s]
123it [00:15,  8.01it/s]
124it [00:15,  8.06it/s]
125it [00:15,  8.05it/s]
126it [00:15,  8.05it/s]
127it [00:15,  8.05it/s]
128it [00:15,  8.05it/s]
129it [00:16,  8.04it/s]
130it [00:16,  8.04it/s]
131it [00:16,  8.05it/s]
132it [00:16,  8.05it/s]
133it [00:16,  8.04it/s]
134it [00:16,  8.05it/s]
135it [00:16,  8.04it/s]
136it [00:16,  8.05it/s]
137it [00:17,  8.04it/s]
138it [00:17,  8.05it/s]
139it [00:17,  8.05it/s]
140it [00:17,  8.04it/s]
141it [00:17,  8.04it/s]

Epoch: 1, Step: 200, Loss: 37.073400209248064



202it [00:25,  8.04it/s]
203it [00:25,  8.03it/s]
204it [00:25,  8.04it/s]
205it [00:25,  8.03it/s]
206it [00:25,  8.03it/s]
207it [00:25,  7.99it/s]
208it [00:25,  8.01it/s]
209it [00:25,  8.04it/s]
210it [00:26,  8.04it/s]
211it [00:26,  8.05it/s]
212it [00:26,  8.00it/s]
213it [00:26,  8.06it/s]
214it [00:26,  8.05it/s]
215it [00:26,  8.05it/s]
216it [00:26,  8.04it/s]
217it [00:26,  8.04it/s]
218it [00:27,  8.04it/s]
219it [00:27,  8.04it/s]
220it [00:27,  8.04it/s]
221it [00:27,  8.04it/s]
222it [00:27,  8.05it/s]
223it [00:27,  8.05it/s]
224it [00:27,  8.05it/s]
225it [00:27,  8.04it/s]
226it [00:28,  8.03it/s]
227it [00:28,  8.04it/s]
228it [00:28,  8.03it/s]
229it [00:28,  7.98it/s]
230it [00:28,  8.03it/s]
231it [00:28,  8.05it/s]
232it [00:28,  8.04it/s]
233it [00:28,  8.05it/s]
234it [00:29,  8.03it/s]
235it [00:29,  8.04it/s]
236it [00:29,  8.03it/s]
237it [00:29,  7.98it/s]
238it [00:29,  8.05it/s]
239it [00:29,  8.05it/s]
240it [00:29,  8.03it/s]
241it [00:29,  8.03it/s]

...

Epoch: 7, Step: 300, Loss: 0.8472936454042792



302it [00:37,  8.02it/s]
303it [00:37,  8.02it/s]
304it [00:37,  8.02it/s]
305it [00:37,  8.02it/s]
306it [00:38,  8.02it/s]
307it [00:38,  8.02it/s]
308it [00:38,  8.02it/s]
309it [00:38,  8.01it/s]
310it [00:38,  8.02it/s]
311it [00:38,  8.00it/s]
312it [00:38,  8.01it/s]
313it [00:38,  8.01it/s]
314it [00:39,  8.02it/s]
315it [00:39,  8.02it/s]
316it [00:39,  8.02it/s]
317it [00:39,  8.02it/s]
318it [00:39,  8.02it/s]
319it [00:39,  8.00it/s]
320it [00:39,  8.03it/s]
321it [00:39,  8.03it/s]
322it [00:40,  8.02it/s]
323it [00:40,  8.02it/s]
324it [00:40,  8.02it/s]
325it [00:40,  7.98it/s]
326it [00:40,  8.04it/s]
327it [00:40,  8.04it/s]
328it [00:40,  8.03it/s]
329it [00:40,  8.03it/s]
330it [00:41,  8.02it/s]
331it [00:41,  8.03it/s]
332it [00:41,  8.02it/s]
333it [00:41,  8.03it/s]
334it [00:41,  8.03it/s]
335it [00:41,  8.03it/s]
336it [00:41,  8.03it/s]
337it [00:41,  8.03it/s]
338it [00:42,  8.02it/s]
339it [00:42,  8.02it/s]
340it [00:42,  8.02it/s]
341it [00:42,  8.03it/s]


Epoch: 7, Test Loss: 0.9361355628275201, Test Accuracy: 0.8346787084398977




0it [00:00, ?it/s]
1it [00:00,  9.92it/s]
2it [00:00,  8.70it/s]
3it [00:00,  8.41it/s]
4it [00:00,  8.24it/s]
5it [00:00,  8.17it/s]
6it [00:00,  8.11it/s]
7it [00:00,  8.08it/s]
8it [00:00,  8.02it/s]
9it [00:01,  8.07it/s]
10it [00:01,  8.07it/s]
11it [00:01,  8.05it/s]
12it [00:01,  8.06it/s]
13it [00:01,  8.04it/s]
14it [00:01,  8.04it/s]
15it [00:01,  8.04it/s]
16it [00:01,  8.01it/s]
17it [00:02,  8.05it/s]
18it [00:02,  8.04it/s]
19it [00:02,  8.04it/s]
20it [00:02,  8.04it/s]
21it [00:02,  8.04it/s]
22it [00:02,  8.04it/s]
23it [00:02,  8.04it/s]
24it [00:02,  8.04it/s]
25it [00:03,  8.04it/s]
26it [00:03,  8.04it/s]
27it [00:03,  8.03it/s]
28it [00:03,  8.03it/s]
29it [00:03,  8.03it/s]
30it [00:03,  8.03it/s]
31it [00:03,  8.03it/s]
32it [00:03,  8.04it/s]
33it [00:04,  8.03it/s]
34it [00:04,  8.04it/s]
35it [00:04,  8.04it/s]
36it [00:04,  8.04it/s]
37it [00:04,  8.03it/s]
38it [00:04,  8.04it/s]
39it [00:04,  8.04it/s]
40it [00:04,  8.04it/s]
41it [00:05,  8.03it/s]
42it 

KeyboardInterrupt: 

# Quantization Test

In [ ]:
import copy

model_quant_naive = copy.deepcopy(model)

pseudo_quantize_model_weight(model_quant_naive, w_bit=4, q_group_size=128)

accuracies = []
losses = []
model_quant_naive.eval()
with torch.no_grad():
  for step, batch in tqdm.tqdm(enumerate(imdb_test_loader)):
    data_fields = ['input_ids', 'attention_mask', 'label']

    for k in data_fields:
      if k == 'label':
        continue
      try:
        batch[k] = torch.stack(batch[k], dim=1)
      except:
        print(batch[k])
        raise ValueError("CLOWN")


    batch = {k: torch.tensor(batch[k]).to(device, non_blocking=True) for k in data_fields}
    y_pred = model_quant_naive(batch['input_ids'], attention_mask=batch['attention_mask'])

    loss = IMDBMetrics.loss(y_pred, batch['label'])
    accuracies.append(IMDBMetrics.accuracy(y_pred, batch['label']).item())
    losses.append(loss.item())


  avg_loss = np.mean(losses)
  avg_accuracy = np.mean(accuracies)
  print()
  print(f"Epoch: {epoch + 1}, Test Loss: {avg_loss}, Test Accuracy: {avg_accuracy}")
  print()

0it [00:00, ?it/s]/tmp/ipython-input-3392332848.py:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch = {k: torch.tensor(batch[k]).to(device, non_blocking=True) for k in data_fields}
391it [00:35, 10.97it/s]


Epoch: 8, Test Loss: 1.105022748870313, Test Accuracy: 0.827141943794992



# Misc